## (3) Same as (2), except using CIFAR dataset instead of MNIST

In [39]:
import torch
import torch.nn as nn
from torchvision import datasets
import torchvision.transforms as transforms
import torch.optim as optim # See https://pytorch.org/docs/stable/optim.html

train_dataset = datasets.CIFAR10(root='./cifar_data/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.CIFAR10(root='./cifar_data/', train=False, transform=transforms.ToTensor(), )

'''
(1) output shape of model: output is 10 instead of 1
(2) Sigmoid --> Softmax
(3) BCELoss --> CrossEntropyLoss
'''

'''
Using GPU
'''
import torch
import torch.optim as optim # See https://pytorch.org/docs/stable/optim.html
from utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")


'''
Define Model
'''
# https://pytorch.org/docs/stable/generated/torch.nn.Module.html
class MLP(nn.Module) :
    def __init__(self, in_dim, out_dim, inter_dim=256) :
        super().__init__()
        self.in_dim = in_dim
        
        self.linear1 = nn.Linear(in_dim, inter_dim, bias=True)
        #self.act1 = nn.ReLU()        
        #self.linear2 = nn.Linear(inter_dim, inter_dim, bias=True)                
        #self.act2 = nn.ReLU()        
        #self.linear3 = nn.Linear(inter_dim, inter_dim, bias=True)  
        self.act3 = nn.ReLU()        
        self.linear4 = nn.Linear(inter_dim, out_dim, bias=True)  
        
    def forward(self, x) :
        z = self.linear1(x)
        #z = self.act1(z)
        #z = self.linear2(z)
        #z = self.act2(z)
        #z = self.linear3(z)  
        z = self.act3(z)   
        z = self.linear4(z)
        return z

'''
Choose loss function
'''
# https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html
loss = torch.nn.CrossEntropyLoss() # (2) Change to CrossEntropyLoss


''' Define model and optimizer'''
MLP_model = MLP(32*32*3, 10) # (4) change input/output shape
# Reverse mode automatic differentiation
OPTIMIZER = optim.SGD(MLP_model.parameters(),  lr=0.1)

'''
DataLoader is used to apply minibatch SGD
'''
batch_size = 128
image_size = 32*32*3
# https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

MLP_model.to(device)

import time
start = time.time()
num_epoch = 50
for epoch in range(num_epoch):
    for image,label in train_loader :

        # Forward process
        hat_y = MLP_model(image.view(-1, 32*32*3).to(device))
        cost = loss(hat_y, label.to(device))

        # Wipe up gradient in the previous step
        OPTIMIZER.zero_grad() 

        # Computes the gradient of current tensor w.r.t. graph leaves.
        cost.backward() 

        # Updates the parameters
        OPTIMIZER.step()
    
    print("{}-epoch/total {}, loss: {:.4f}".format(epoch, num_epoch, cost))

end = time.time()
print("Training Time: {:.2f}".format(end-start))



MLP_model.to("cpu")

''' Test for train_dataset '''

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
count = 0
for image,label in train_loader :
    label = label
    prediction = torch.argmax(MLP_model(image.view(len(image),3*32*32)),dim=1)
    count += (prediction == label).sum()
        
print('Test for train_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(train_dataset.targets), count/len(train_dataset.targets) * 100. ) )    

''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(MLP_model(image.view(len(image),3*32*32)),dim=1)
    count += (prediction == label).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

total_params = sum(p.numel() for p in MLP_model.parameters())
print("Number of Parameters: ", total_params)

Files already downloaded and verified
0-epoch/total 50, loss: 1.9045
1-epoch/total 50, loss: 1.8542
2-epoch/total 50, loss: 1.6636
3-epoch/total 50, loss: 1.7690
4-epoch/total 50, loss: 1.4166
5-epoch/total 50, loss: 1.8641
6-epoch/total 50, loss: 1.7540
7-epoch/total 50, loss: 1.4854
8-epoch/total 50, loss: 1.4054
9-epoch/total 50, loss: 1.2799
10-epoch/total 50, loss: 1.7573
11-epoch/total 50, loss: 1.3779
12-epoch/total 50, loss: 1.2421
13-epoch/total 50, loss: 1.6388
14-epoch/total 50, loss: 1.5338
15-epoch/total 50, loss: 1.4585
16-epoch/total 50, loss: 1.3079
17-epoch/total 50, loss: 1.1140
18-epoch/total 50, loss: 1.2155
19-epoch/total 50, loss: 1.2771
20-epoch/total 50, loss: 1.2431
21-epoch/total 50, loss: 1.1023
22-epoch/total 50, loss: 1.2858
23-epoch/total 50, loss: 1.0891
24-epoch/total 50, loss: 1.3368
25-epoch/total 50, loss: 1.1885
26-epoch/total 50, loss: 1.4121
27-epoch/total 50, loss: 1.0974
28-epoch/total 50, loss: 1.2249
29-epoch/total 50, loss: 1.4617
30-epoch/tot

# --- Test Code ---

In [41]:
''' Test for test_dataset '''
count = 0
wrong_answer = []
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
for image,label in test_loader :
    label = label
    prediction = torch.argmax(MLP_model(image.view(len(image),image_size)),dim=1)
    count += (prediction == label).sum()
    
#     else:
#         wrong_answer += [i]   
        
print('Test for test_dataset: correctly matched/total: {}/{} accuracy: {:.2f}%\n'.format(count, len(test_dataset.targets), count/len(test_dataset.targets) * 100. ) )    

Test for test_dataset: correctly matched/total: 4991/10000 accuracy: 49.91%



# --- Review ---

### layer 수를 2개이상으로 올리니 정확도 미묘하게 내려감.
### batch_size를 128이상으로 올리니 정확도 내려감.
### (batch_size를 올린 만큼 epoch 수를 올려야 하지만 학습 시간이 오래걸려 test 불가)
### batch = 128, epoch = 50으로 마무리
### inter_dim 각각 256이상, 미만은 정확도 내려감